In [1]:
import cv2
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation

In [2]:
# https://www.bilibili.com/video/BV1DF4m1u74U P6 35:43-35:48
# download tool: IDM
video_path = "./target_video2.mp4"
video_start_min =  35
video_start_sec =  43
video_end_min =  35
video_end_sec =  48
output_csv_name = 'target2_coord.csv'
output_fig_path = './frames2'
output_video_name ='output_video2.mp4'

frame_number_range = [70, 75]

In [3]:
if not os.path.exists(output_fig_path):
    os.makedirs(output_fig_path)

In [4]:
video = cv2.VideoCapture(video_path)

In [5]:
fps = video.get(cv2.CAP_PROP_FPS)
print('FPS：',  fps)

FPS： 30.0


In [6]:
# target frame
# 目标帧数
start_frame = int((video_start_min*60+video_start_sec)*fps)
end_farme = int((video_end_min*60+video_end_sec)*fps)

In [7]:
 frame_count = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
print('Total frame：',  frame_count)

Total frame： 108000


In [8]:
frame_width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
print('Size:', frame_width, '*', frame_height)

Size: 1920 * 1080


In [9]:
# 32:30-32:34

In [10]:
# click event function
# 点击事件函数
# https://blog.csdn.net/weixin_41788560/article/details/96306156
def click_event(event, x, y, flags, param):
    if event == cv2.EVENT_RBUTTONDOWN:
        frame_clicked.append(temp_frame)
        frame_list.append(frame.copy())
    if event == cv2.EVENT_LBUTTONDOWN:
        # print(x, y)
        font = cv2.FONT_HERSHEY_SIMPLEX
        strXY = str(x) + ', '+ str(y)
        mouse_position_x.append(x)
        mouse_position_y.append(y)
        cv2.destroyAllWindows()

In [11]:
df = pd.read_csv(output_csv_name)

In [12]:
x = np.array(df['x']).astype(int)
y = np.array(df['y']).astype(int)

In [13]:
# output image
# 输出图片
temp_frame = start_frame
while temp_frame <= end_farme:
    video.set(cv2.CAP_PROP_POS_FRAMES, temp_frame)
    
    ret, frame = video.read()
    if ret:
        frame = np.array(frame)
        
        # BGR2RGB
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        im = cv2.circle(frame, [x[temp_frame - start_frame],y[temp_frame - start_frame]], 5, [255, 0, 0],2)
        
        im =cv2.putText(im, str(temp_frame - start_frame), [20, 70], cv2.FONT_HERSHEY_SIMPLEX, 2., (255, 225, 0), 3)
        
        for i in range(0, temp_frame - start_frame):
            im = cv2.line(im, [x[i],y[i]], [x[i+1],y[i+1]], [255, 225, 0],1)
            if i >= frame_number_range[0] and i <= frame_number_range[1]: 
                im = cv2.putText(im, str(i+1), [x[i+1],y[i+1] - 40], cv2.FONT_HERSHEY_SIMPLEX, .8, (255, 225, 0), 2)
        plt.imsave(os.path.join(output_fig_path, str(temp_frame - start_frame) + '.png'), im)
    temp_frame +=1
    

In [14]:
# output video
# 输出视频
writer = cv2.VideoWriter(output_video_name, cv2.VideoWriter_fourcc(*"mp4v"), fps, (frame_width, frame_height))
temp_frame = start_frame

while temp_frame <= end_farme:
    video.set(cv2.CAP_PROP_POS_FRAMES, temp_frame)
    
    ret, frame = video.read()
    if ret:
        frame = np.array(frame)
        
        # BGR2RGB
        # frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        im = cv2.circle(frame, [x[temp_frame - start_frame],y[temp_frame - start_frame]], 5, [255, 0, 0],2)
        for i in range(0, temp_frame - start_frame):
            im = cv2.line(frame, [x[i],y[i]], [x[i+1],y[i+1]], [255, 225, 0],1)
        writer.write(im)
    temp_frame +=1
writer.release()

In [15]:
video.release()